# Assignment 5: Solving PDEs with Fourier Analysis

We have been discussing the solutions of ordinary and partial differential equations, as well as the concepts of Fourier Series and Transforms.

The goal of this homework is to put all of that to work. 

## Table of Contents

1. [Heat equation](#The-Heat-Equation)
1. [Finite difference solution](#Finite-difference-to-solving-the-heat-equation)
1. [Spectral analysis solution](#Spectral-analysis-approach-to-solving-the-heat-equation)
1. Exercises
    1. [Exercise 1](#Exercise-1)
    1. [Exercise 2](#Exercise-2)

## The Heat Equation

A basic fact of nature is that heat flows from hot to cold, that is, from regions of high temper- ature to regions of low temperature. We give these words mathematical expression by stating that the rate of heat flow $\vec{H}$ through a material is proportional to the gradient of the temperature $T$ across the material:

$$
\vec{H} = -K \nabla T (x, t),
$$

where $K$ is the thermal conductivity of the material. The total amount of heat $Q(t)$ in the material at any one time is proportional to the integral of the temperature over the material’s volume

$$
Q(t) = \int d\vec{x} C \rho(\vec{x})T(\vec{x},t)
$$

where $C$ is the specific heat of the material and $\rho$ is its density. Because energy is conserved, the rate of decrease in $Q$ with time must equal the amount of heat flowing out of the material. After this energy balance is struck and the divergence theorem applied, there results the heat equation:

$$
\frac{\partial T(\vec{x},t)}{\partial t} = \frac{K}{C\rho}\nabla^2 T(\vec{x},t)
$$

The heat equation is a parabolic PDE with space and time as independent variables. The specification of this problem implies that there is no temperature variation in directions perpendicular to the bar ($y$ and $z$), and so we have only one spatial coordinate in the Laplacian:

$$
\frac{\partial T(\vec{x},t)}{\partial t} = \frac{K}{C\rho}\frac{\partial^2 T(\vec{x},t)}{\partial x^2}
$$

For this problem, the initial temperature of the bar and the boundary conditions are $T(x,t=0)=100C$, $T(x=0,t)=T(x=L,t)=0C$.

## Finite difference approach to solving the heat equation

As we did with Laplace’s equation, the numerical solution is based on converting the differential equation to a finite-difference ("difference") equation. We discretize space and time on a lattice and solve for solutions on the lattice sites. The horizontal nodes with white centers correspond to the known values of the temperature for the initial time, while the vertical white nodes correspond to the fixed temperature along the boundaries. If we also knew the temperature for times along the bottom row, then we could use a relaxation algorithm as we did for Laplace’s equation. However, with only the top row known, we shall end up with an algorithm that steps forward in time one row at a time, as in the children’s game leapfrog.

As is often the case with PDEs, the algorithm is customized for the equation being solved and for the constraints imposed by the particular set of initial and boundary conditions. With only one row of times to start with, we use a forward-difference approximation for the time derivative of the temperature:

$$
\frac{\partial T(x,t)}{\partial t} \simeq \frac{ T(x, t+\Delta t) - T(x,t) }{\Delta t}
$$

Because we know the spatial variation of the temperature along the entire top row and the left and right sides, we are less constrained with the space derivative as with the time derivative.

Consequently, as we did with the Laplace equation, we use the more accurate central-difference approximation for the space derivative:

$$
\frac{\partial^2 T(x,t)}{\partial x^2} \simeq \frac{ T(x+\Delta x, t) + T(x - \Delta x,t) - 2 T(x,t)}{\Delta x^2}
$$

Substitution of these approximations into the original expression yields the heat difference equation

$$
\frac{ T(x, t+\Delta t) - T(x,t) }{\Delta t} = \frac{K}{C \rho} \frac{ T(x+\Delta x, t) + T(x - \Delta x,t) - 2 T(x,t)}{\Delta x^2}
$$

We reorder this last expression into a form in which $T$ can be stepped forward in time $t$: 

$$
T_{i,j+1} = T_{i,j} + \eta [T_{i+1,j} + T_{i-1,j} - 2 T_{i,j}]
$$

and

$$
\eta = \frac{K\Delta t}{C\rho\Delta x^2},
$$

where $x = i\Delta x$ and $t = j\Delta t$. 

This algorithm is explicit because it provides a solution in terms of known values of the temperature. If we tried to solve for the temperature at all lattice sites simultaneously, then we would have an implicit algorithm that requires us to solve equations involving unknown values of the temperature. 

We see that the temperature at space-time point $(i, j+1)$ is computed from the three temperature values at an earlier time $j$ and at adjacent space values $i \pm 1, i$. We start the solution at the top row, moving it forward in time for as long as we want and keeping the temperature along the ends fixed at 0 K .


## Spectral analysis approach to solving PDEs

An additional way to solve PDEs in an efficient way other than multigrid methods is by using spectral methods which are based on expressing the solution as a Fourier series. 

The basic idea behind the Fourier series is to express a function as a sum of simpler functions. The Fourier series uses cosine and sine functions as its basis functions, often expressed with exponential functions. The periodic function $f$ is associated with the Fourier series

$$
f(x) = \sum_{k=-\infty}^{\infty} c_k e^{i 2 \pi k x/L}
$$

where $c_k$ are the Fourier coefficients, $k$ represents the frequency and $L$ the spatial grid length. It is the Fourier coefficients that need to be calculated in order to satisfy the equality above. To find these coefficients we use the essential orthogonality properties of the exponential functions.

Such an approach is called the [Spectral Method](https://en.wikipedia.org/wiki/Spectral_method)

One perspective we can take to formulate spectral methods is one where we want to reconstruct/deconstruct periodic functions $y(x)$ of some wave-length $L$, based on its values at the discrete set of $N$ equally spaced points
$$
    x_n = \frac{(n-1) L}{N}.
$$

Consider the Fourier coefficients of our function $y(x)$,
$$
    \hat{y~}_m = \frac{1}{L} \int^L_0 y(x) \exp(-i k(m) x) dx, \quad m = 0, \pm 1, \pm 2, \ldots
$$
where the wave number associated with the Fourier coefficient $m$ is
$$
    k(m) = \frac{2 \pi m}{L}.
$$
From these coefficients we can reconstruct the function $y(x)$ as
$$
    y(x) = \sum^\infty_{m=-\infty} \hat{y~}_m \exp(i k(m) x).
$$

### Example code for Fourier Transform

The discrete Fourier transform is efficiently computed by the Fast Fourier transform algorithm. For a real function $I(x)$, the relevant code for computing and plotting the discrete Fourier transform appears in the example below.

In [ ]:
def I(x):
    return sin(2*pi*x) + 0.5*sin(4*pi*x) + 0.1*sin(6*pi*x)

# Mesh
L  = 10; Nx = 100
x  = np.linspace(0, L, Nx+1)
dx = L/float(Nx)

# Discrete Fourier transform
A = np.fft.rfft(I(x))
A_amplitude = np.abs(A)

# Compute the corresponding frequencies
freqs = np.linspace(0, pi/dx, A_amplitude.size)

plt.plot(freqs, A_amplitude)
plt.show()

## Exercise 1

1. Write a function to solve the heat equation.
1. Define a 2-D array `T[101][2]` for the temperature as a function of space and time. The first index is for the 100 space divisions of the bar, and the second index is for present and past times (because you may have to make thousands of time steps, you save memory by saving only two times).
1. For time $t = 0 (j = 1)$, initialize $T$ so that all points on the bar except the ends are at 100 K. Set the temperatures of the ends to 0 K.
1. Apply the forward-difference approximation for the time derivative of the temperature to obtain the temperature at the next time step.
1. Assign the present-time values of the temperature to the past values: `T[i][1] = T[i][2], i = 1,... , 101.`
1. Start with 50 time steps. Once you are confident the program is running properly, use thousands of steps to see the bar cool smoothly with time. For approximately every 500 time steps, print the time and temperature along the bar.

Some things that you'll want to check, verify:
1. Check that your program gives a temperature distribution that varies smoothly along the bar and agrees with the boundary conditions
1. Check that your program gives a temperature distribution that varies smoothly with time and attains equilibrium. You may have to vary the time and space steps to obtain well- behaved solutions.
1. Make surface plots of temperature versus position for several times.

Optional:
1. Make a surface plot of temperature versus position versus time.
1. Plot the isotherms (contours of constant temperature).
1. Compare the analytic and numeric solutions (and the wall times needed to compute them). If the solutions differ, suspect the one that does not appear smooth and continuous.

## Exercise 2

In class we discussed how to implement a Fourier approximation for the sawtooth wave. Now, I want you to do the same for the following:

1. Square wave, with coefficients falling off as $\mathcal{O}(|M|^{-1})$ (Exact $\hat{y~}_M = -2 i / (\pi M), M$ odd)
1. Function $(1-0.6 \cos(2 \pi x / L))^{-1}$, with coefficients falling off *spectrally* (beyond any algebraic order) (Exact $\hat{y~}_M = 1.25 \cdot 3^{-|M|}$)
1. Function $5 \sin(2 \pi x / L) + 2 \cos(3 \pi x / L) + \sin(5 \pi x / L)$ and decompose this into its components; then check that there are three of them in the ratio 5:2:1 (or 25:4:1 if a power spectrum is plotted) and that they resum to give the input signal.

You should find that the smoother the function the fewer Fourier coefficients we need to have hanging around in order for us to approximate the functions $y(x)$.